In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from   PIL import Image
from   torch.utils import data
from   torchvision import transforms
%matplotlib inline

##### 自定义ResNet的Block

In [2]:
class ResnetBasicBlock(nn.Module):
    
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(
                               in_channels,   # 输入图像通道数
                               out_channels,  # 输出图像通道数
                               kernel_size=3, # kernel=3 需要设置padding=1 复原原来图像大小
                               padding=1,     # 将图像的大小复原，在卷积后的图像周围加一圈像素
                               bias=False     # 由于后面需要进行BatchNormalize操作，bias会消失，为了节省操作的时间，不在这里添加bias
        )
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
                               in_channels,   # 输入图像通道数
                               out_channels,  # 输出图像通道数
                               kernel_size=3, # kernel=3 需要设置padding=1 复原原来图像大小
                               padding=1,     # 将图像的大小复原，在卷积后的图像周围加一圈像素
                               bias=False     # 由于后面需要进行BatchNormalize操作，bias会消失，为了节省操作的时间，不在这里添加bias
        )
        self.bn2   = nn.BatchNorm2d(out_channels)
        
    def forward(self,x):
        residual = x                          # 保留一部分x作为residual，用来参与计算残差                  
        out      = self.conv1(x)
        out      = self.bn1(out)
        out      = F.relu(out,inplace=True)     # 设置就地修改out的值
        out      = self.conv2(out)
        out      = self.bn2(out)
        out     += residual
        out      = F.relu(out)
        return  out

##### 查看内置ResNet网络形状

In [3]:
model = torchvision.models.resnet34         # 查看resnet网络的源代码

In [4]:
model = torchvision.models.resnet34